In [1]:
!pip install -U yfinance

In [19]:
import yfinance as yf
import pandas as pd
import numpy as np


# Net Income (순이익) 추출 함수
def get_net_income_from_operatings(stock):
    financials = stock.quarterly_financials.T
    return financials["Net Income Continuous Operations"]

# Operating Cash Flow (영업활동 현금흐름) 추출 함수
def get_operating_cashflow(stock):
    cashflow = stock.quarterly_cashflow.T
    return cashflow["Cash Flow From Continuing Operating Activities"]

def get_total_asset(stock):
    quarterly_balancesheet = stock.quarterly_balancesheet.T
    return quarterly_balancesheet['Total Assets']

def get_net_income(stock):
    financials = stock.quarterly_financials.T
    return financials['Net Income']

# Cash Dividends Paid (배당금 지급) 추출 함수
def get_cash_dividends_paid(stock):
    try:
        cashflow = stock.quarterly_cashflow.T
        return cashflow.get("Cash Dividends Paid", pd.Series()).fillna(0)
    except AttributeError:
        return pd.Series()

# Repurchase of Capital Stock (자사주 매입) 추출 함수
def get_repurchase_of_stock(stock):
    try:
        cashflow = stock.quarterly_cashflow.T
        return cashflow.get("Repurchase Of Capital Stock", pd.Series()).fillna(0)
    except AttributeError:
        return pd.Series()


# 모든 데이터를 종합하는 함수
def get_financial_data(ticker):
    stock = yf.Ticker(ticker)

    # 개별 함수 호출하여 데이터 추출
    net_income = get_net_income(stock)
    cash_dividends_paid = get_cash_dividends_paid(stock)
    repurchase_of_stock = get_repurchase_of_stock(stock)
    net_income_from_operatings = get_net_income_from_operatings(stock)
    cashflow_of_operating = get_operating_cashflow(stock)
    total_asset = get_total_asset(stock)

    # 데이터프레임 생성
    df = pd.DataFrame({
        'Net Income': net_income,
        'Cash Dividends Paid': cash_dividends_paid,
        'Repurchase of Capital Stock': repurchase_of_stock,
        'Net Income from Operating':net_income_from_operatings,
        'Cashflow of Operating':cashflow_of_operating,
        'Shareholder Return': cash_dividends_paid + repurchase_of_stock,
        '영업 활동으로 발생하는 현금 흐름 (Income/Operating) %': (net_income_from_operatings/cashflow_of_operating)*100,
        '자산 대비 수익 창출 능력 (Income/Assets) %': (net_income_from_operatings/total_asset)*100
    })

    return df

In [20]:
# 예시 종목: 애플 (AAPL)
ticker = "GOOG"
df = get_financial_data(ticker)

<ipython-input-19-ec25af08b929>:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return cashflow.get("Cash Dividends Paid", pd.Series()).fillna(0)
<ipython-input-19-ec25af08b929>:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return cashflow.get("Repurchase Of Capital Stock", pd.Series()).fillna(0)


In [21]:
df['영업 활동으로 발생하는 현금 흐름 (Income/Operating) %']

,영업 활동으로 발생하는 현금 흐름 (Income/Operating) %
2023-06-30,NaN
2023-09-30,NaN
2023-12-31,109.368226
2024-03-31,82.023017
2024-06-30,88.65991
2024-09-30,85.676591
2024-12-31,67.844451


In [22]:
df['자산 대비 수익 창출 능력 (Income/Assets) %']

,자산 대비 수익 창출 능력 (Income/Assets) %
2023-06-30,NaN
2023-09-30,NaN
2023-12-31,5.141007
2024-03-31,5.808764
2024-06-30,5.694481
2024-09-30,6.11273
2024-12-31,5.893536


In [23]:
df/np.power(10,9)

,Net Income,Cash Dividends Paid,Repurchase of Capital Stock,Net Income from Operating,Cashflow of Operating,Shareholder Return,영업 활동으로 발생하는 현금 흐름 (Income/Operating) %,자산 대비 수익 창출 능력 (Income/Assets) %
2023-06-30,NaN,0.000,0.000,NaN,NaN,0.000,NaN,NaN
2023-09-30,NaN,0.000,0.000,NaN,NaN,0.000,NaN,NaN
2023-12-31,20.687,0.000,-16.191,20.687,18.915,-16.191,0.0,0.0
2024-03-31,23.662,0.000,-15.696,23.662,28.848,-15.696,0.0,0.0
2024-06-30,23.619,0.000,-15.684,23.619,26.64,-15.684,0.0,0.0
2024-09-30,26.301,-2.455,-15.291,26.301,30.698,-17.746,0.0,0.0
2024-12-31,26.536,-2.442,-15.551,26.536,39.113,-17.993,0.0,0.0
